# Units mystery in the Priestley-Taylor equation 

Author: Stan Schymanski, stan.schymanski@datascience.ch

In this worksheet, we explore the famous Priestley-Taylor equation (Priestley and Taylor, 1972) using the Environmental Science for SageMath (essm) package, downloadable from https://github.com/environmentalscience/essm.

In [1]:
from essm.equations import Equation
from essm.variables import Variable
from essm.variables.utils import generate_metadata_table
from essm.variables.units import markdown, joule, kelvin, kilogram, meter, pascal, second

Below, we use essm `Variable` class to define relevant variables used in the paper by Priestley and Taylor. Since units were not provided in the paper, we assume standard SI units for each variable. Since $q_s$ was referred to as specific humidity at saturation by Priestley and Taylor (1972), we assume units of kg m$^{-3}$. Consequently, since $s=\partial q_s/\partial T$, the units of $s$ are assumed to be kg m$^{-3}$ K$^{-1}$.

In [2]:
class E_w(Variable):
    """Wet surface evaporation
    (E in Priestley and Taylor, 1972))
    """
    unit = kilogram/meter^2/second
    latex_name = 'E'
    
class H_w(Variable):
    """Wet surface sensible heat flux (positive outwards)"""
    unit = joule/second/meter^2
    latex_name = 'H'
      
class q_a(Variable):
    """Specific humidity"""
    unit = kilogram/meter^3
    
class q_s(Variable):
    """Specific humidity at saturation"""
    unit = kilogram/meter^3
    
class s_PT(Variable):
    """Priestley-Taylor $\partial q_s/\partial T$"""
    unit = kilogram/meter^3/kelvin
    latex_name = 's'
    
class T(Variable):
    """Temperature"""
    units = kelvin

We will import latent heat of evaporation and specific heat of dry air from the essm thermodynamics library:

In [3]:
from essm.variables.physics.thermodynamics import lambda_E, c_pa

We will re-define `lambda_E` as `L_E` to have the same latex representation as in the paper:

In [4]:
class L_E(lambda_E.definition):
    """Latent heat"""
    
    latex_name = 'L'
    default = lambda_E.definition.default
    unit = lambda_E.definition.unit

In [5]:
table(list(generate_metadata_table([E_w, H_w, L_E, c_pa, s_PT])))

Symbol,Name,Description,Default value,Units
,c_pa,Specific heat of dry air.,1010.0,J K kg
,E_w,"Wet surface evaporation (E in Priestley and Taylor, 1972))",-,kg m s
,H_w,Wet surface sensible heat flux (positive outwards),-,J m s
,L_E,Latent heat,2450000.0,J kg
,s_PT,Priestley-Taylor,-,kg K m


Eq. 3 was given in the paper as:
$$ \frac{LE}{H} = \frac{L}{c_p} s $$

Using the variables defined above, we can write it as:

In [6]:
eq3 = L_E*E_w/H_w == L_E/c_pa*s_PT
eq3.show()

E_w*L_E/H_w == L_E*s_PT/c_pa

Now, we will try to define the above expression as a physical equation, using our `Equation` class:

In [7]:
try: 
    class eq_3(Equation):
        """Equation 3 in Priestley and Taylor (1972)"""
        expr = L_E*E_w/H_w == L_E/c_pa*s_PT
except Exception, error:
    print error

Invalid expression units: 1 == kilogram/meter^3


Since $EL/H$ is non-dimensional, the right hand side of Eq. 3 must be non-dimensioanl, too. Clearly, the units of Eq. 3 do not match if we use our assumptions about the units of $L$, $s$ and $c_{pa}$. Either the equation is missing a division by a density term (units of kg m$^{-3}$) on the right hand side, or one of our assumptions about the units involved was different to what the authors had in mind. 